## Data Collection Code from MAST

In [24]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
import pandas as pd

In [25]:
#Open and read the unmatched pantheon dataset
df = pd.read_csv('unmatched_pantheon.csv')
vp = pd.read_csv('valid_points_index_list.csv')
print(vp.dtypes)

0    int64
dtype: object


In [26]:
i = 12
print(df['resolved_coord'][i]) 

obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][i],
                                        radius="0.006 deg",
                                        intentType = 'science',
                                        filters = ['F1*'],
                                        #calibration = 1,
                                        obs_collection=['HST', 'JWST'])
obs_table = obs_table[obs_table['calib_level']==3]
print(len(obs_table))
print(obs_table)

211.65662 -5.43942
19
intentType obs_collection provenance_name ...   objID1       distance     
---------- -------------- --------------- ... --------- ------------------
   science           JWST         CALJWST ... 759738914                0.0
   science           JWST         CALJWST ... 760489567                0.0
   science           JWST         CALJWST ... 760557180                0.0
   science            HST         HAP-SVM ... 653163140                0.0
   science            HST         HAP-SVM ... 657100566                0.0
   science            HST         HAP-SVM ... 658287973                0.0
   science            HST         HAP-SVM ... 658460871                0.0
   science            HST         HAP-SVM ... 658925460                0.0
   science            HST         HAP-MVM ... 662996461 12.680915717477268
   science            HST         HAP-MVM ... 662996467 12.760854502984333
   science            HST         HAP-MVM ... 663002042                0.0
   

In [27]:
#Creating a function that will create a folder for the data (named the resolved coordinate) and keep the query there

def save_images_supernova(resolved_coord_index):
    """
    If there are images for the supernovae in JWST or HST at the resolved query, create a folder and keep the data there
    Parameters:
        resolved_coord_index: integer, the index number of the query
    Returns:
        Folder of Data at requested file path (pantheon_data_folder/{resolved coord})
    """
    print(resolved_coord_index, df['resolved_coord'][resolved_coord_index])

    #try the the query, if a error is thrown then print "No Data Points" and "skip" this data point, else make folder and store data
    try:
        obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            filters = ['F1*'],
                                            obs_collection=['HST', 'JWST'])
        obs_table = obs_table[obs_table['calib_level']==3]
        data_products = Observations.get_product_list(obs_table)
    except: 
        print("No Data Points :(")
        exit
    else:
        print("Has Data Points!")
        folder_path = "pantheon_data_folder/{}".format(df['SNID'][resolved_coord_index])
        os.makedirs(folder_path, exist_ok=True)
        manifest = Observations.download_products(data_products, download_dir=folder_path, extension=['fits'])
        print(manifest)

In [28]:
for i in range(len(vp['0'])):
    save_images_supernova(vp['0'][i])

12 211.65662 -5.43942
Has Data Points!
INFO: 5846 of 6876 products were duplicates. Only returning 1030 unique product(s). [astroquery.mast.utils]


KeyboardInterrupt: 